## **Generating Datasets from a single image**

We have an image of a banknote and whant to generate a dataset from it that later will be used for training an autoencoder. This should reppresent clean banknotes.

In the following we randomly apply **border crop, scaling, rotation, contrast change**.

In [1]:
import cv2
import numpy as np
import os
from PIL import Image
import utils

In [40]:
def generate_augmented_images(original_image_path, output_dir, output_resolution, num_images):
    """ Generate a dataset of augmented images from an original image.

    Args:
        original_image_path (_type_): _description_
        output_dir (_type_): _description_
        output_resolution (_type_): _description_
        num_images (_type_): _description_
    """
    output_dir_train = os.path.join(output_dir, 'train/good')
    output_dir_test_good = os.path.join(output_dir, 'test/good')
    output_dir_anomaly = os.path.join(output_dir, 'test/anomaly/')
    output_dir_gt = os.path.join(output_dir, 'ground_truth/anomaly/')

    os.makedirs(output_dir_train, exist_ok=True)
    os.makedirs(output_dir_test_good, exist_ok=True)
    os.makedirs(output_dir_anomaly, exist_ok=True)
    os.makedirs(output_dir_gt, exist_ok=True)

    original_image = cv2.imread(original_image_path)
    
    # ----------------------------------------------------------------------------
    for i in range(num_images[0]):

        new_image = utils.split_and_stack(original_image)
        new_image = utils.augment(new_image, output_resolution)
        output_path = os.path.join(output_dir_train, f'{str(i).zfill(5)}.png')
        cv2.imwrite(output_path, new_image)

    print(f"Generated {num_images[0]} images in '{output_dir_train}'.")

    # ----------------------------------------------------------------------------
    for i in range(num_images[1]):

        new_image = utils.split_and_stack(original_image)
        new_image = utils.augment(new_image, output_resolution)
        output_path = os.path.join(output_dir_test_good, f'{str(i).zfill(5)}.png')
        cv2.imwrite(output_path, new_image)

    print(f"Generated {num_images[1]} images in '{output_dir_test_good}'.")

    # ----------------------------------------------------------------------------
    for i in range(num_images[2]):
    
        new_image = utils.split_and_stack(original_image)
        new_image = utils.augment(new_image, output_resolution)

        scribbles_text = random.choice(["Erlangen", "0123456789", "Scribbles", "Prof. Lampe", "HS-Mittweida", "M.Draghi", "50 Euro"])
        new_image, mask = MyScribbles(new_image, scribbles_text = scribbles_text)

    
     
        mask = cv2.bitwise_not(mask)
        mask = cv2.cvtColor(mask, cv2.COLOR_BGR2GRAY)
        _, mask = cv2.threshold(mask, 1, 255, cv2.THRESH_BINARY)

        # Save the modified image to the dataset directory
        output_path = os.path.join(output_dir_anomaly, f'{str(i).zfill(5)}.png')
        if not cv2.imwrite(output_path, new_image):
            raise Exception("Could not write image")
    
        output_path = os.path.join(output_dir_gt, f'{str(i).zfill(5)}_mask.png')
        if not cv2.imwrite(output_path, mask):
            raise Exception("Could not write image")


    print(f"Generated {num_images[2]} images in '{output_dir_anomaly}'.")
    print(f"Generated {num_images[2]} images in '{output_dir_gt}'.")

    

In [41]:
original_image_path = '50euro_front.jpg'
output_dir = 'augmented_front/'     # folder/subfolder/subsubfolder/
output_resolution = (1024, 1024)
num_images = [14, 12, 10]	        # Number of images per class: [train, test_good, test_bad]

generate_augmented_images(original_image_path, output_dir, output_resolution, num_images)


Generated 14 images in 'augmented_front/train/good'.
Generated 12 images in 'augmented_front/test/good'.
['fonts\\journal.ttf']
['fonts\\journal.ttf']
['fonts\\journal.ttf']
['fonts\\journal.ttf']
['fonts\\journal.ttf']
['fonts\\journal.ttf']
['fonts\\journal.ttf']
['fonts\\journal.ttf']
['fonts\\journal.ttf']
['fonts\\journal.ttf']
Generated 10 images in 'augmented_front/test/anomaly/'.
Generated 10 images in 'augmented_front/ground_truth/anomaly/'.


We aslo want some banknotes with anomalies. The library [augraphy](https://github.com/sparkfish/augraphy) can be used for it.

In [3]:
from augraphy import *
import random
from glob import glob
import numpy as np
from ink2 import InkGenerator

# Applies the Augmentation to input data.
def MyScribbles(image, 
                layer=None, 
                mask=None, 
                keypoints=None, 
                bounding_boxes=None, 
                force=False,
                fonts_directory = "fonts/",
                scribbles_type="text",
                scribbles_ink="pen",
                scribbles_location="random",
                scribbles_size_range=(300, 400),
                scribbles_count_range=(1, 1),
                scribbles_thickness_range=(1, 1),
                scribbles_brightness_change=[32, 64, 128],
                scribbles_skeletonize=0,
                scribbles_skeletonize_iterations=(2, 3),
                scribbles_color=(0,0,0),
                scribbles_text = random.choice(["Erlangen", "0123456789", "Scribbles", "Prof. Lampe", "HS-Mittweida"]),
                
                scribbles_text_rotate_range=(-30, 30),
                scribbles_lines_stroke_count_range=(1, 10),
                p=1):
    
    # ink_type: Types of ink, select from "pencil", "pen", "marker" or "highlighter"
    
    fonts_list = glob(fonts_directory + "/*.ttf")
    #print(fonts_list)

    if scribbles_type == "random":
        scribbles_type = random.choice(["lines", "texts"])
    else:
        scribbles_type = scribbles_type

    if scribbles_ink == "random":
        scribbles_ink = random.choice(["pencil", "pen", "marker", "highlighter"])
    else:
        scribbles_ink = scribbles_ink

    if scribbles_skeletonize == "random":
        scribbles_skeletonize = random.choice([0, 1])
    else:
        scribbles_skeletonize = scribbles_skeletonize

    if scribbles_color == "random":
        scribbles_color = (random.randint(0, 255), random.randint(0, 255), random.randint(0, 255))
    else:
        scribbles_color = scribbles_color

    if scribbles_location != "random":
        ysize, xsize = image.shape[:2]
        target_x, target_y = scribbles_location
        # check if provided location is float and scale them with target size
        if target_x >= 0 and target_x <= 1 and isinstance(target_x, float):
            target_x = int(target_x * xsize)
        if target_y >= 0 and target_y <= 1 and isinstance(target_y, float):
            target_y = int(target_y * ysize)
        scribbles_location = (target_x, target_y)
    else:
        scribbles_location = scribbles_location

    # create an ink generator and generate scribbles
    ink_generator = InkGenerator(
                ink_type=scribbles_ink,
                ink_draw_method=scribbles_type,
                ink_draw_iterations=scribbles_count_range,
                ink_location=scribbles_location,
                ink_background=image,
                ink_background_size=None,
                ink_background_color=None,
                ink_color=scribbles_color,
                ink_min_brightness=0,
                ink_min_brightness_value_range=(0, 0),
                ink_draw_size_range=scribbles_size_range,
                ink_thickness_range=scribbles_thickness_range,
                ink_brightness_change=scribbles_brightness_change,
                ink_skeletonize=scribbles_skeletonize,
                ink_skeletonize_iterations_range=scribbles_skeletonize_iterations,
                ink_text=scribbles_text,
                ink_text_font=fonts_list,
                ink_text_rotate_range=scribbles_text_rotate_range,
                ink_lines_coordinates="random",
                ink_lines_stroke_count_range=scribbles_lines_stroke_count_range,
            )

    img_size = image.shape[0:2]

    img = np.zeros((1024, 1024, 3), dtype = "uint8")

    image_output, mask_output = ink_generator.generate_ink()

    return image_output, mask_output

In [48]:
generate_augmented_images(original_image_path, output_dir, output_resolution, num_images)

Generated 14 images in 'augmented_front/train/good'.
Generated 12 images in 'augmented_front/test/good'.
['fonts\\journal.ttf']
['fonts\\journal.ttf']
['fonts\\journal.ttf']
['fonts\\journal.ttf']
['fonts\\journal.ttf']
['fonts\\journal.ttf']
['fonts\\journal.ttf']
['fonts\\journal.ttf']
['fonts\\journal.ttf']
['fonts\\journal.ttf']
Generated 10 images in 'augmented_front/test/anomaly/'.
Generated 10 images in 'augmented_front/ground_truth/anomaly/'.


In [2]:
# Rename files
folder_path = 'real_front/train/good/'
utils.rename_files(folder_path)

Done!


##### Iterates over a directory creating new images from existing ones. Outputs the images with scribbles and their binary masks.

In [59]:
directory_path = 'real_front/test/good/'

# Define the output directory for the dataset
output_dir_anomaly = 'real_front/test/anomaly/'
output_dir_gt = 'real_front/ground_truth/anomaly/'
# Create the output directory if it doesn't exist
os.makedirs(output_dir_anomaly, exist_ok=True)
os.makedirs(output_dir_gt, exist_ok=True)


# Define the resolution for the resulting images
output_resolution = (1024, 1024)


i = 0
for filename in os.listdir(directory_path):
    i += 1
    print(filename)
    
    image = cv2.imread(directory_path + filename)
    #image = cv2.rotate(image, cv2.ROTATE_180)
    #image = split_and_stack(image)
    #image = cv2.resize(image, output_resolution)


    scribbles_text = random.choice(["0123456789", "Scribbles", "Prof. Lampe", "HS-Mittweida", "M.Draghi", "50 Euro"])
    image, mask = MyScribbles(image, scribbles_text = scribbles_text)

    
     
    mask = cv2.bitwise_not(mask)
    mask = cv2.cvtColor(mask, cv2.COLOR_BGR2GRAY)
    _, mask = cv2.threshold(mask, 1, 255, cv2.THRESH_BINARY)

    # Save the modified image to the dataset directory
    output_path = os.path.join(output_dir_anomaly, f'{str(i).zfill(5)}.png')
    if not cv2.imwrite(output_path, image):
        raise Exception("Could not write image")
    
    output_path = os.path.join(output_dir_gt, f'{str(i).zfill(5)}_mask.png')
    if not cv2.imwrite(output_path, mask):
        raise Exception("Could not write image")


print(f"Generated {i} images.")


00001.png
00002.png
00003.png
00004.png
00005.png
00006.png
00007.png
00008.png
00009.png
00010.png
00011.png
00012.png
00013.png
00014.png
00015.png
Generated 15 images.


In [8]:
# resize and rotate only

directory_path = 'real_back/excluded/'
output_resolution = (1024, 1024)


for filename in os.listdir(directory_path):

    img_path = os.path.join(directory_path, filename)
    image = cv2.imread(img_path)
    image = cv2.rotate(image, cv2.ROTATE_180)
    image = cv2.resize(image, output_resolution)
    
    if not cv2.imwrite(img_path, image):
        raise Exception("Could not write image")
    

print(f"Done.")

Done.


In [15]:
#utils.resize_and_rotate((1024, 1024), 'EUR50_1/back/bad/', 'EUR50_1/back/bad/transformed/')

Done.
